In [1]:
from heapy.data.reduction import gbmTTE
from heapy.data.xselect import epXselect
from heapy.data.retrieve import gbmRetrieve
from heapy.util.time import fermi_utc_to_met

In [2]:
utc = '2024-02-19T06:21:42'

In [3]:
fermi_met = fermi_utc_to_met(utc)
gbm_rtv = gbmRetrieve.from_utc(utc=utc, t1=-400, t2=400)

gbm_tte = gbmTTE(gbm_rtv.rtv_res['tte']['n9'])
gbm_tte.timezero = fermi_met
gbm_tte.time_slice(-200, 300)
gbm_tte.energy_filter(10, 1000)

gbm_event = gbm_tte.to_event
gbm_event.lc_t1t2 = [-200, 300]
gbm_event.lc_binsize = 2

In [4]:
wxt_event = epXselect.from_wxtobs('ep13600003859wxtCMOS44l23v2', 's2')

wxt_event.src_evtfile = '../wxt/curve/src.evt'
wxt_event.bkg_evtfile = '../wxt/curve/bkg.evt'

wxt_event.timezero = utc
wxt_event.lc_t1t2 = [-200, 300]
wxt_event.lc_binsize = 2

In [5]:
from matplotlib import rcParams
import matplotlib.pyplot as plt

rcParams['font.family'] = 'serif'
rcParams['font.sans-serif'] = 'Georgia'
rcParams['font.size'] = 12
rcParams['pdf.fonttype'] = 42

fig, ax = plt.subplots(1, 1, figsize=(8, 5))
p1, = ax.plot(gbm_event.lc_time, gbm_event.net_rate, 
              lw=1.5, c='r', label='Fermi/GBM light curve')
ax.set_xlim([-200, 300])
ax.set_ylim([-36, 144])
ax.set_xlabel(f'Time since {utc} (s)')
ax.set_ylabel('Fermi/GBM counts per sencond')
ax.minorticks_on()
ax.tick_params(axis='x', which='both', direction='in', labelcolor='k', colors='k')
ax.tick_params(axis='y', which='both', direction='in', labelcolor='k', colors='k')
ax.tick_params(which='major', width=1.0, length=5)
ax.tick_params(which='minor', width=1.0, length=3)
ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')

ax1 = ax.twinx()
p2, = ax1.plot(wxt_event.lc_time, wxt_event.net_rate, 
               lw=1.5, c='b', label='EP-WXT light curve')
ax1.set_ylabel('EP-WXT counts per sencond')
ax1.set_ylim([-1, 4])

plt.legend(handles=[p1, p2], frameon=False)
fig.savefig('./joint_curve.pdf', bbox_inches='tight', pad_inches=0.1, dpi=300)
plt.close(fig)

In [10]:
from heapy.temporal.lag import Lag

lag = Lag(gbm_event.src_cts, wxt_event.src_cts, gbm_event.bkg_cts, wxt_event.bkg_cts*wxt_event.regratio, 
          dt=2, xtype='pg', ytype='pp', xbcts_se=gbm_event.bkg_cts_err)
lag.mc_simulation(nmc=1000)
lag.callag(width=20, deg=4)
lag.save('./spectral_lag')


+-----------------------------------------------+
 lag (s)        lag_le (s)     lag_he (s)     
 5.1445         4.9363         5.7665         
+-----------------------------------------------+

